In [1]:
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

%matplotlib inline


In [2]:
file = 'titanic_data.csv'
full_data = pd.read_csv(file)
full_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
full_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [4]:
y = full_data['Survived']

X = full_data.drop(['Cabin', 'Survived', 'Name'], axis = 1)

In [5]:
X = pd.get_dummies(X)

In [6]:
train_X, test_X, train_y, test_y = train_test_split(X, y, random_state = 1)

In [7]:
pipeline = make_pipeline(SimpleImputer(), DecisionTreeClassifier())


In [8]:
pipeline.fit(train_X, train_y)

Pipeline(memory=None,
     steps=[('simpleimputer', SimpleImputer(copy=True, fill_value=None, missing_values=nan, strategy='mean',
       verbose=0)), ('decisiontreeclassifier', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impur...      min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))])

In [9]:
predictions = pipeline.predict(test_X)

In [10]:
acc = accuracy_score(test_y, predictions)

In [44]:
print(acc)

0.8026905829596412
